In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/content/mushrooms.csv')
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g


We see that all the data is categorical data , let's transform the categorical data into numerical

In [3]:
from sklearn.preprocessing import LabelEncoder
new_df = df.apply(LabelEncoder().fit_transform)

In [4]:
new_df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,0,3,2,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,0,2,2,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,0,2,2,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,0,3,2,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,1,3,2,2,7,7,0,2,1,0,3,0,1


##Start of Neural network function

In [25]:
#Activation functions
def sigmoid(x):
  return 1/(1+np.exp(-x))

def sigmoid_derivative(x):
  return sigmoid(x)*(1-sigmoid(x))

def softmax(x):
    e_x = np.exp(x)
    softmax = e_x / e_x.sum(axis=1, keepdims=True)
    return softmax

def relu(x):
  return np.maximum(0,x)

def relu_derivative(x):
  x[x <=0] = 0
  x[x>0]=1
  return x

### Back Propogation

In [26]:
def back_prop(inputs, hw, ow, hb, ob, learning_rate, ah, y):
 
    # softmax gradient
    softmax_der = (y - outputs) / outputs.shape[0]
    
    # output layer weights and biases gradients
    ow_der = np.dot(ah.T, softmax_der)
    ob_der = np.sum(softmax_der, axis = 0, keepdims = True)
    
    # hidden layer weights and biases gradients
    ah_der = np.dot(softmax_der, ow.T)
    ah_der[ah <= 0] = 0
    hw_der = np.dot(inputs.T, ah_der)
    hb_der = np.sum(ah_der, axis = 0, keepdims = True)
    
    # update output layer and hidden layer weights and biases
    ow -= learning_rate * ow_der
    hw -= learning_rate * hw_der
    
    ob -= learning_rate * ob_der 
    hb -= learning_rate * hb_der

### Feed Forward and Model Training




In [27]:
def mlp(inputs, outputs, hidden_layer_width, alpha, epochs):
    
    d = dict()
    np.random.seed(42)

    hw = np.random.rand(len(inputs[0]), hidden_layer_width)
    ow = np.random.rand(hidden_layer_width, 7)  

    # intialize biases for hidden and output layers to zeros
    hb = np.zeros((1, hidden_layer_width))
    ob = np.zeros((1, 7))

    # learning rate
    learning_rate = alpha


    for epoch in range(1, epochs + 1):
        # hidden layer
        zh = np.dot(inputs, hw) + hb
        ah = relu(zh)
    
        #feed forward using activation function
        zo = np.dot(ah, ow) + ob
        ao = softmax(zo)
    
        # calculate mean squared error
        diff = ao - outputs
        mse = (np.square(diff)).mean(axis=0)
        
        # backpropagation
        back_prop(inputs, hw, ow, hb, ob, alpha, zh, ah, zo, diff)
    
        # save the final weights and biases of the model for testing
        if epoch == epochs:
            print("After %d epochs, mean squared error is: %.2f" % (epoch,mse[0]))
            d['hidden layer weights'] = hw
            d['output layer weights'] = ow
            d['hidden layer biases'] = hb
            d['output layer biases'] = ob
            
    return d

### Model Accuracy

In [28]:
def model_accuracy(test_inputs, test_outputs, hidden_weights, output_weights, hidden_biases, output_biases):
    # hidden layer
    zh = np.dot(test_inputs, hidden_weights) + hidden_biases
    ah = relu(zh)
    
    # output layer
    zo = np.dot(ah, output_weights) + output_biases
    ao = softmax(zo)

    # test model accuracy

    preds_correct_boolean =  np.argmax(ao, 1) == np.argmax(test_outputs, 1)
    correct_predictions = np.sum(preds_correct_boolean)
    accuracy = correct_predictions / len(test_outputs)

    print("Accuracy is: %f" % accuracy)
   


### Train and Test for inputs in neural netowrk

In [29]:
# inputs and outputs
inputs = new_df.drop(columns=['class'])
outputs = new_df['class'].values

# 75-25 split for training and testing sets
split = round(.75 * len(inputs))

X_train = np.array(inputs[:split])
X_test = np.array(inputs[split:])

y_train = np.array(outputs[:split])
y_test = np.array(outputs[split:])

y_train = y_train.reshape(len(y_train),1)
y_test = y_test.reshape(len(y_test),1)

### Run model


In [31]:
model = mlp(inputs=X_train, outputs=y_train, hidden_layer_width=40, alpha=0.05, epochs=10000)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


After 10000 epochs, mean squared error is: nan


### Model accuracy

In [32]:
hw = model['hidden layer weights']
ow = model['output layer weights']
hb = model['hidden layer biases']
ob = model['output layer biases']

model_accuracy(test_inputs=X_test, test_outputs=y_test, hidden_weights=hw, output_weights=ow, 
               hidden_biases=hb, output_biases=ob)

Accuracy is: 1.000000
